In [161]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [162]:
# Load the data
members = pd.read_csv('members_filtered.csv')
transactions = pd.read_csv('transactions_filtered.csv')
user_logs = pd.read_csv('user_logs_filtered.csv')
labels = pd.read_csv('labels_filtered.csv')
#user_logs.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,AtRE9/xHjNrX8tcuUb/cdBRvUHpwXhZrWY9E9sk1tDk=,20160615,26,4,2,7,88,66,22906.055
1,f3HbYjEEVBVeWcP3FNDCLKJXpuIZPvX0oFyn4XLMN3k=,20151225,22,2,1,1,143,135,37190.485
2,Q7nR6DbodHwUTGPi5Z0yOsdC2cSOhiWIe4cNEVmnJ9I=,20160404,24,7,0,2,67,79,15816.946
3,Qx9cihonlt2hkBa3VHNj+nwX2QelZXpyHnEAIjD199w=,20160923,24,5,1,2,92,115,24879.848
4,mf3zFfyeZLl8zEWhV+GyWQYjt3orfQUCcOIMwBJ60+k=,20160430,61,5,4,5,98,132,24534.754


In [163]:
def pd_to_date(df_col):
    df_col = pd.to_datetime(df_col, format = '%Y%m%d')
    return df_col

#Convert to date
user_logs['date'] = pd_to_date(user_logs['date'])
#user_logs.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,AtRE9/xHjNrX8tcuUb/cdBRvUHpwXhZrWY9E9sk1tDk=,2016-06-15,26,4,2,7,88,66,22906.055
1,f3HbYjEEVBVeWcP3FNDCLKJXpuIZPvX0oFyn4XLMN3k=,2015-12-25,22,2,1,1,143,135,37190.485
2,Q7nR6DbodHwUTGPi5Z0yOsdC2cSOhiWIe4cNEVmnJ9I=,2016-04-04,24,7,0,2,67,79,15816.946
3,Qx9cihonlt2hkBa3VHNj+nwX2QelZXpyHnEAIjD199w=,2016-09-23,24,5,1,2,92,115,24879.848
4,mf3zFfyeZLl8zEWhV+GyWQYjt3orfQUCcOIMwBJ60+k=,2016-04-30,61,5,4,5,98,132,24534.754


High level plan.  Let's try several things:

* Initially get a table by user (1 row per user)
* User most recent date (max date)
* User first date (min date)
* How long they've been listening:  Min vs. max date by user
* Matrix of all the following (cartesian product)
    * Total X=(seconds, 100, 985, 75, 50, 25, unique), avg per day of X, maybe median per day of X
    * Last day, last 7 days, last 30 days, last 90, 180, 365, total (note last day is relative to user)
    * % change in periods above vs. outside that period.
    * ??? Total change in periods above vs. outside that period.
    * ??? Variation (do they listen consistently, or is it varied)?

In [164]:
#Create our groupby user object 
user_logs_gb = user_logs.groupby(['msno'], sort=False)

In [165]:
#This cell is slow

#Append max date to every row in main table
user_logs['max_date'] = user_logs_gb['date'].transform('max')
user_logs['days_before_max_date'] = (user_logs['max_date'] - user_logs['date']).apply(lambda x: x.days)
    #The .apply(lambda...  just converts it from datetime to an integer, for easier comparisons later.

#Generate user's first date, last date, and tenure
#Also, the user_logs_features table will be the primary table to return from the transactions table
user_logs_features = (user_logs_gb
    .agg({'date':['max', 'min', lambda x: max(x) - min(x)]})
    .rename(columns={'max': 'max_date', 'min': 'min_date','<lambda>':'listening_tenure'})
                      )

#Add a 3rd level, for joining data later
user_logs_features = pd.concat([user_logs_features], axis=1, keys=['date_features'])

In [166]:
# Total X=(seconds, 100, 985, 75, 50, 25, unique), avg per day of X, maybe median per day of X
# Last day, last 7 days, last 30 days, last 90, 180, 365, total (note last day is relative to user)
    
for num_days in [7, 14, 31, 90, 180, 365, 999]:
    #Create groupby object for items with x days
    ul_gb_xdays = (user_logs.loc[(user_logs['days_before_max_date'] < num_days)]
                   .groupby(['msno'], sort=False))

    #Generate sum and mean (and count, once) for all the user logs stats
    past_xdays_by_user = (ul_gb_xdays
        .agg({'num_unq':['sum', 'mean', 'count'],
              'total_secs':['sum', 'mean'],
              'num_25':['sum', 'mean'],
              'num_50':['sum', 'mean'],
              'num_75':['sum', 'mean'],
              'num_985':['sum', 'mean'],
              'num_100':['sum', 'mean'],
             })
                      )
    #Append level header
    past_xdays_by_user = pd.concat([past_xdays_by_user], axis=1, keys=['within_days_' + str(num_days)])

    #Join (append) to user_logs_features table
    user_logs_features = user_logs_features.join(past_xdays_by_user, how='inner')

In [160]:
#Next, let's look at changes in last 7 days vs. last 30 days, and last 30 days vs. last 180 days.

#Also, need to think about users with < x days tenure.

In [167]:
#Save to CSV
user_logs_features.to_csv('user_logs_features.csv')

High level plan.  Let's try several things:

* Initially get a table by user (1 row per user)
* User most recent date (max date)
* User first date (min date)
* How long they've been listening:  Min vs. max date by user
* Matrix of all the following (cartesian product)
    * Total X=(seconds, 100, 985, 75, 50, 25, unique), avg per day of X, maybe median per day of X
    * Last day, last 7 days, last 30 days, last 90, 180, 365, total (note last day is relative to user)
    * % change in periods above vs. outside that period.
    * ??? Total change in periods above vs. outside that period.
    * ??? Variation (do they listen consistently, or is it varied)?

# Other stuff below.  Delete for final product.

In [ ]:
#Sample code, for reference:       
    #Gets # of column levels
        #past_xdays_by_user.columns.nlevels
    
    #This gets only items <= 7 days old
        #user_logs.loc[(user_logs['days_before_max_date'] <= 7)]

    #Get all records from specfic user in past 45 days
        #user_logs.loc[(user_logs['msno'] == 'aof/9XT0zVdONwrq7vd+V4y3saluJQy+Wj0dlo9TKvI=') & (user_logs['days_before_max_date'] < 45)]

    #Just count songs
        #user_logs_gb.agg({'num_unq':{'count','sum'}}).head()
        
    #This code resolves columns with the same name, but shouldn't be used
        #user_logs_features = user_logs_features.join(past_xdays_by_user, lsuffix='_l', rsuffix='_r', how='inner')

    #This code doesn't error but the results aren't as expected 
        # user_logs_gb.apply(lambda x: pd.Series(dict(
        #     song_count=(x.days_before_max_date <= 7).count(),
        #     song_sum=(x.days_before_max_date <= 7).sum()
        # )))
        
#Ideas
    #Maybe get avg. days_before_max_date ... maybe something here.

In [16]:
# months = {
#     1 : 'January',
#     2 : 'February',
#     3 : 'March',
#     4 : 'April',
#     5 : 'May',
#     6 : 'June',
#     7 : 'July',
#     8 : 'August',
#     9 : 'September',
#     10 : 'October',
#     11 : 'November',
#     12 : 'December'
# }
# transaction_count = {}
# for key, value in months.items():
#     transaction_count[key] = len(transactions[pd.DatetimeIndex(transactions['transaction_date']).month == key])
# plt.bar(list(transaction_count.keys()), transaction_count.values())
# plt.xticks(list(months.keys()), months.values(), rotation = 'vertical')
# plt.title('Transaction Count by Month')
# plt.show()

In [ ]:
pd.DatetimeIndex(transactions['transaction_date']).month